In [ ]:
import os

In [ ]:
os.chdir('..')

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [2]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [12]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,param_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(param_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training=self.config.training
        prepare_base_model=self.config.prepare_base_model
        params=self.params
        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chest-CT-Scan-data")
        create_directories([training.root_dir])

        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [4]:
import tensorflow as tf
import time

In [16]:
class Training:
    def __init__(self,config: TrainingConfig):
        self.config=config

    def get_base_model(self):
        model=tf.keras.models.load_model(self.config.updated_base_model_path)
        return model
    
    def train_valid_generator(self):
        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.2
        )
        dataflow_kwargs=dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator=valid_datagenerator

        self.train_generator=train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs

        )

    @staticmethod
    def save_model(path:Path,model:tf.keras.Model):
        model.save(path)
        print(f"Model saved at {path}")

    def train(self):
        self.steps_per_epoch=self.train_generator.samples // self.config.params_batch_size
        self.validation_steps=self.valid_generator.samples // self.config.params_batch_size
        
        model=self.get_base_model()
        model.fit(
            self.train_generator,
            steps_per_epoch=self.steps_per_epoch,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            epochs=self.config.params_epochs
        )
        self.save_model(
            path=self.config.trained_model_path,
            model=model
        )




In [7]:
import os

os.chdir('..')

In [17]:
try:
    config=ConfigurationManager()
    training_config=config.get_training_config()
    training=Training(config=training_config)
    training.train_valid_generator()
    training.train()
    print("Model trained successfully.")
except Exception as e:
    raise e

{2025-08-21 12:00:32,904: INFO: common: yaml file: config\config.yaml loaded successfully}
{2025-08-21 12:00:32,920: INFO: common: yaml file: params.yaml loaded successfully}
{2025-08-21 12:00:32,926: INFO: common: created directory at: artifacts_root}
{2025-08-21 12:00:32,940: INFO: common: created directory at: artifacts/training}


Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/2
17/17 [==============================] - 561s 32s/step - loss: 0.9337 - accuracy: 0.5483 - val_loss: 0.6915 - val_accuracy: 0.6094
Epoch 2/2
17/17 [==============================] - 486s 29s/step - loss: 0.6909 - accuracy: 0.5753 - val_loss: 0.6893 - val_accuracy: 0.6094
Model saved at artifacts\training\trained_model.h5
Model trained successfully.
